In [1]:
from selenium import webdriver
import time, pandas as pd, numpy as np, os
from selenium.webdriver.common.by import By

from dotenv import load_dotenv

load_dotenv()


local = os.environ.get("RUN")
print(local)

from local import options


MAIN_URL  = 'https://cuantoestaeldolar.pe'


if local == 'local':
    driver = webdriver.Chrome()
else:
    driver = webdriver.Chrome(options=options)

driver.get(MAIN_URL)

# sunat

a = driver.find_element(By.XPATH, '//*[@id="converter"]/div/div[2]/div[1]')

print(a.text)
# firsm

local
Sunat
3.780
0.00
3.789
0.00


In [2]:
import numpy as np

In [3]:
div_elements = driver.find_elements(By.CSS_SELECTOR, "div.flex.my-auto a")

# Itera sobre los elementos encontrados y extrae el atributo href de cada uno
hrefs = [element.get_attribute("href") for element in div_elements]

In [4]:
texto_numerico_element = driver.find_element(By.CSS_SELECTOR, 'p.ValueQuotation_text___mR_0')
texto_numerico = texto_numerico_element.text.strip()
print("Texto numérico:", texto_numerico)

Texto numérico: 


In [5]:
# left div
range_left = list(range(26, 38))
range_right = list(range(2, 14))

# a
a= '//*[@id="__next"]/main/div[3]/div[5]/div/div[1]/div[26]/div[1]/div[3]/a'
# //*[@id="__next"]/main/div[3]/div[5]/div/div[1]/div[37]/div[1]/div[3]/a

# //*[@id="__next"]/main/div[3]/div[5]/div/div[2]/div[2]/div[1]/div[3]/a
# //*[@id="__next"]/main/div[3]/div[5]/div/div[2]/div[13]/div[1]/div[3]/a



left_divs = [
    f'//*[@id="__next"]/main/div[3]/div[5]/div/div[1]/div[{n}]/div[1]'
    for n in range_left
]
left_img = [
    f'//*[@id="__next"]/main/div[3]/div[5]/div/div[1]/div[{n}]/div[1]/div[1]/span/img'
    for n in range_left
]

left_a = [
    f'//*[@id="__next"]/main/div[3]/div[5]/div/div[1]/div[{n}]/div[1]/div[3]/a'
    for n in range_left
]



right_divs = [
    f'//*[@id="__next"]/main/div[3]/div[5]/div/div[2]/div[{n}]/div[1]'
    for n in range_right
]
right_img = [
    f'//*[@id="__next"]/main/div[3]/div[5]/div/div[2]/div[{n}]/div[1]/div[1]/span/img'
    for n in range_right
]

right_a = [
    f'//*[@id="__next"]/main/div[3]/div[5]/div/div[2]/div[{n}]/div[1]/div[3]/a'
    for n in range_right
]


divs = left_divs + right_divs
imgs = left_img + right_img
a_s = left_a + right_a

In [12]:
def extract_info_div(
        text_path, img, url
):
    dolar = driver.find_element(By.XPATH, text_path).text.split("\n")
    source_img = driver.find_element(By.XPATH, img).get_attribute("src")
    url_ref = driver.find_element(By.XPATH, url).get_attribute("href")
    return dolar + [source_img, url_ref]
import pandas as pd
a= [
    extract_info_div(t, i, u) 
    for t, i, u in zip(divs, imgs, a_s)
]
import re
def extract_domain(url):
    match = re.search(r'https?://(www\.)?([^/]+)', url)
    if match:
        domain = match.group(2).split('.')[0]
        return domain
    return None
data = pd.DataFrame(a, columns = ['trash', 'compra', 'compra_cambio', 'venta', 'venta_cambio', 'url_img', 'url'])
data['dominio'] = data['url'].apply(extract_domain)
data.sample(2)

,trash,compra,compra_cambio,venta,venta_cambio,url_img,url,dominio
19,Cambiar,3.800,0.00,3.820,0.00,https://s3-ced-uploads-01.s3.amazonaws.com/171...,https://perudolar.pe/?utm_source=ced&utm_mediu...,perudolar
11,Cambiar,0.000,-3.795,0.000,-3.828,https://s3-ced-uploads-01.s3.amazonaws.com/170...,https://cambiomas.com.pe/?utm_source=ced,cambiomas


In [13]:
html = driver.page_source

In [14]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')

In [15]:

data = []

# Encontrar todos los bloques que contienen la información
blocks = soup.find_all('div', class_='flex py-4 justify-between h-[72px]')

for block in blocks:
    img_tag = block.find('img', alt=True)
    img_url = img_tag['src'] if img_tag else None

    values = block.find_all('p', class_='text-2xl md:w-40 flex justify-end')
    if len(values) >= 2:
        compra = values[0].text
        venta = values[1].text
    else:
        compra = None
        venta = None

    data.append({
        'img_url': img_url,
        'compra': compra,
        'venta': venta
    })

print(data)

[{'img_url': 'data:image/svg+xml,%3csvg%20xmlns=%27http://www.w3.org/2000/svg%27%20version=%271.1%27%20width=%27100%27%20height=%2730%27/%3e', 'compra': '3.729', 'venta': '3.901'}, {'img_url': 'data:image/svg+xml,%3csvg%20xmlns=%27http://www.w3.org/2000/svg%27%20version=%271.1%27%20width=%27100%27%20height=%2730%27/%3e', 'compra': '3.725', 'venta': '3.860'}, {'img_url': 'data:image/svg+xml,%3csvg%20xmlns=%27http://www.w3.org/2000/svg%27%20version=%271.1%27%20width=%2780%27%20height=%2730%27/%3e', 'compra': '3.721', 'venta': '3.901'}]


In [16]:
pd.DataFrame(data)

,img_url,compra,venta
0,"data:image/svg+xml,%3csvg%20xmlns=%27http://ww...",3.729,3.901
1,"data:image/svg+xml,%3csvg%20xmlns=%27http://ww...",3.725,3.860
2,"data:image/svg+xml,%3csvg%20xmlns=%27http://ww...",3.721,3.901
